<a href="https://colab.research.google.com/github/Shreyash007/deep_learning_course/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 1 CS6910**
Shreyash Gadgil (ED22S016)

In [2]:
!pip install --upgrade wandb
!wandb login aa5afea12b4fda1e7f8310b597eb17c73d1176d2 #my API key for wandb login 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 27.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 22.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=a3b6eedd981c389545a1a2f72bef2e2c52f0f5a4c7bad7e57a83ba0dd0a6d09b
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
wandb: 

Import libraries

In [3]:
from keras.datasets import fashion_mnist
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import wandb

# **Q1 Solution**
Labelling dataset

In [4]:
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
#importing dataset

#checking size of training and test dataset
print("Training dataset shape, X=",X_train.shape,", Y=",Y_train.shape)
print("Test dataset shape, X=",X_test.shape,", Y=",Y_test.shape)

n_img_classes=len(np.unique(Y_train)) #to find out number of unique images 
img_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] #taken from keras documentation


def show_images(n_img_classes,img_labels,X,Y):

   setImgs = []
   setLabels=[]

   for i in range(len(X)):
     if len(setImgs)==n_img_classes:
       break
     if img_labels[Y[i]] not in setLabels:
      setImgs.append(X[i])
      setLabels.append(img_labels[Y[i]])

   run=wandb.init(project='CS-6910 A1',entity='shreyashgadgil007',reinit=True)
   run.log({"Sample from each class":list(wandb.Image(img,caption= caption) for img,caption in zip(setImgs,setLabels))})
   run.finish()

#--------------------------------------------------------------------------Q1 SOLUTION-----------------------------------------------------------------------------------------------
#show_images(n_img_classes,img_labels,X_test,Y_test)


4422102/4422102 [==============================] - 0s 0us/step
Training dataset shape, X= (60000, 28, 28) , Y= (60000,)
Test dataset shape, X= (10000, 28, 28) , Y= (10000,)


# **Preprocessing dataset for operations**
1. Creating training,test,validation split for dataset
2. Reshaping image matrix and normalizing pixel values
3. One hot encoding classifications


In [35]:
def dataset_preprocess():
  #change the code below to accept different dataset
  (X_1, Y_1), (X_test, Y_test) = fashion_mnist.load_data()
  
  #importing dataset again and normalizing
  X_1 = X_1.reshape(X_1.shape[0],-1)/255.0
  X_test = X_test.reshape(X_test.shape[0],-1)/255.0

  #training and validation split as specified in the question 10%
  X_train, X_val, Y_train, Y_val= train_test_split(X_1,Y_1,test_size=0.1,random_state=0)
  
  #one hot encoding
  Y_train_encoded=np.zeros((Y_train.shape[0],10))
  for i in range(len(Y_train)):
    Y_train_encoded[i][Y_train[i]]=1

  Y_val_encoded=np.zeros((Y_val.shape[0],10))
  for i in range(len(Y_val)):
    Y_val_encoded[i][Y_val[i]]=1

  Y_test_encoded=np.zeros((Y_test.shape[0],10))
  for i in range(len(Y_test)):
    Y_test_encoded[i][Y_test[i]]=1

  return X_train,X_test,X_val,Y_train,Y_val,Y_test,Y_train_encoded,Y_val_encoded,Y_test_encoded

#Below variables will be used in the entire code
X_train,X_test,X_val,Y_train,Y_val,Y_test,Y_train_encoded,Y_val_encoded,Y_test_encoded=dataset_preprocess()


2
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


# **Helper functions**
1. Declared activation functions
2. Declared initialisation functions
3. Declared a function to initialize weights and biases

In [6]:
#---------------------------------------------------------------ACTIVATION FUNCTIONS AND THEIR GRADIENTS-------------------------------------------------------------------------------
def relu(X):
  return max(X,0)

def grad_relu(X):
  return X>0

def sigmoid(X):
  return 1/(1+np.exp(-X))

def grad_sigmoid(X):
  return sigmoid(X)(1-sigmoid(X))

def tanh(X):
  return (np.exp(X)-np.exp(-X))/(np.exp(X)+np.exp(-X))

def grad_tanh(X):
  return 1-(tanh(X))**2

def softmax(X):
  # Calculates the softmax function
  e_X = np.exp(X - np.max(X, axis = 0))
  return e_X / e_X.sum(axis = 0)

def softmax_cross_entropy(Y,Y_pred):
  return -(Y - Y_pred) 

#---------------------------------------------------------------------------LOSS FUNCTION---------------------------------------------------------------------------------------------
def cross_entropy():
  return

def squared_error_loss():
  return

#----------------------------------------------------------------------INITIALISATION FUNCTIONS---------------------------------------------------------------------------------------
def random_initialisation(shape):
  # Initialising a random matrix with given dimensions (shape) as tuple
  np.random.seed(0)
  return np.random.randn(*shape)*0.1

def xavier_initialisation(shape):
    # Calculate the number of inputs and outputs
    n_in = shape[0]
    n_out = shape[1]
    
    # Calculate the variance of the normal distribution
    variance = 2.0 / (n_in + n_out)
    
    # Initialize the weights with random values drawn from the normal distribution
    np.random.seed(0)
    weights = np.random.randn(n_in, n_out) * np.sqrt(variance)
    
    return weights

#------------------------------------------------------------------INITIALIZING WEIGHTS AND BIASES------------------------------------------------------------------------------------
def initialize_w_b(input_layer,hidden_layer,output_layer,init):
  weights=[]
  biases=[]
  layers=[input_layer]+hidden_layer+[output_layer]
  for i in range(len(hidden_layer)+1):
     if init=='random':
       weights.append(random_initialisation((layers[i+1],layers[i])))
     if init=='xavier':
       weights.append(xavier_initialisation((layers[i+1],layers[i])))
     biases.append(np.random.randn(layers[i+1],1))
  return weights, biases

#testing weights and biases output(used for debugging shape size of matrix) 
output_layer = 10
input_layer = X_test.shape[1]
print(X_test.shape)
hidden_layer = [64, 64]
weights, biases = initialize_w_b(input_layer, hidden_layer, output_layer, 'xavier')
for i in range(len(weights)):
    print(weights[i].shape)
    
for i in range(len(biases)):
    print(biases[i].shape)



(10000, 784)
(64, 784)
(64, 64)
(10, 64)
(64, 1)
(64, 1)
(10, 1)


# **Neural Network**
1. With forward propagation
2. Backward propagation
3. Weight updates w.r.t different functions

In [15]:
class NeuralNet():

  def __init__(self,input_layer,hidden_layer,output_layer,initialisation_func,act_function,loss_func):
     self.input_layer=input_layer
     self.hidden_layer=hidden_layer
     self.output_layer=output_layer
     self.initialisation_func=initialisation_func
     self.act_function=act_function
     self.loss_func=loss_func
     self.weights,self.biases = initialize_w_b(self.input_layer,self.hidden_layer,self.output_layer,self.initialisation_func)

  def forward_propagation(self,X):
     #pre-activation
     self.A=[]
     #post-activation
     self.H=[]
     l=0
     
     #pre-activation and post-activation for input layer and first hidden layer
     self.A.append(np.matmul(self.weights[l],X)+self.biases[l])
     if self.act_function=='sigmoid':
       self.H.append(sigmoid(self.A[l]))
     elif self.act_function=='relu':
       self.H.append(relu(self.A[l]))
     elif self.act_function=='tanh':
       self.H.append(tanh(self.A[l]))
     

     #pre-activation and post-activation between hidden layers
     for l in range(1,len(self.hidden_layer)):
       self.A.append(np.matmul(self.weights[l],self.H[l-1])+self.biases[l])
       if self.act_function=='sigmoid':
        self.H.append(sigmoid(self.A[l]))
       elif self.act_function=='relu':
        self.H.append(relu(self.A[l]))
       elif self.act_function=='tanh':
        self.H.append(tanh(self.A[l]))
       
     #pre-activation and post-activation between last hidden layer and output layer
     l=len(self.hidden_layer)
     self.A.append(np.matmul(self.weights[l],self.H[l-1])+self.biases[l])
     self.H.append(softmax(self.A[l]))

     return self.H[-1]
  
def back_propagation(self,X,Y):
        #Initialisng gradient parameters
        #Takes input image and its label and returns the gradient parameters i.e. gradient of weight and gradient of biases
        g_a  = [0]*(len(self.hidden_layer)+1)
        g_h  = [0]*(len(self.hidden_layer)+1)
        g_w  = [0]*(len(self.weights))
        g_b  = [0]*(len(self.biases))

        n_samples = X.shape[1]  # represents batch size


        for l in reversed(range(len(self.hidden_layer)+1)):
            if l == len(self.hidden_layer):
              #change below to functions for calls
              if self.loss_function == 'cross_entropy':
                  g_a[l] = self.H[l]  - Y  
              elif self.loss_function == 'square_loss': 
                  g_a[l] = (self.H[l] - Y) * self.H[l] * (1 - self.H[l]) 
                
            else:
                g_h[l] = (1/n_samples)*np.matmul(self.weights[l+1].T,g_a[l+1])
                if self.activation == 'sigmoid':
                  g_a[l] = (1/n_samples)*np.multiply(g_h[l],grad_sigmoid(self.A[l]))
                elif self.activation == 'tanh':
                  g_a[l] = (1/n_samples)*np.multiply(g_h[l],grad_tanh(self.A[l]))
                elif self.activation == 'relu':
                  g_a[l] = (1/n_samples)*np.multiply(g_h[l],grad_relu(self.A[l]))
                

            if l == 0:
                g_w[l] = (1/n_samples)*np.matmul(g_a[l],X_train.T) 
            else:
                g_w[l] = (1/n_samples)*np.matmul(g_a[l],self.H[l-1].T)

            g_b[l]  = (1/n_samples)*np.sum(g_a[l], axis=1, keepdims = True)
        return g_w,g_b


Q1 Forward propagation solved

In [25]:
q1    = NeuralNet(X_train.shape[1],[64,64,64], 10,'xavier','sigmoid','cross_entropy')
A=q1.forward_propagation(X_train.T)
A=A.T
print(A[0])
print((X_train.T).shape[0])
print(Y_train[0])
print(Y_train_encoded[0])

[0.05723773 0.55586946 0.05466115 0.02037356 0.00672244 0.00941655
 0.1432002  0.06383334 0.01125722 0.07742835]
784
9
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
